In [20]:
import os

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

# install findspark using pip
!pip install -q findspark

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [21]:
# Connect to Postgres 
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-01-27 03:22:40--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.16MB/s    in 0.2s    

2022-01-27 03:22:40 (5.16 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



## Extraction

In [22]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETLProjectAnalysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [24]:
from pyspark import SparkFiles

# Importing electronic reviews
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

elec_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
elec_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [25]:
# Importing sports reviews
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

sport_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Sports_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
sport_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [26]:
# Concatinate two dataframes 
df = elec_df.union(sport_df)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   41409413|R2MTG1GCZLR2DK|B00428R89M|     112201306|yoomall 5M Antenn...|     Electronics|          5|            0|          0|   N|                Y|          Five Stars|       As described.| 2015-08-31|
|         US|   49668221|R2HBOEM8LE9928|B000068O48|     734576678|Hosa GPM-103 3.5m...|     Electronics|          5|    

In [27]:
# Count number of rows
df.count()

7944229

In [28]:
# Drop any nulls
df = df.dropna()


In [29]:
# Check the shape of the dataframe
print((df.count(), len(df.columns)))

(7942659, 15)


In [30]:
# Check data types
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



## Transformation

In [31]:
# to fit review_id_table
review_id_df = df.select('review_id', 'customer_id', 'product_id', 'product_parent', 'review_date')
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2MTG1GCZLR2DK|   41409413|B00428R89M|     112201306| 2015-08-31|
|R2HBOEM8LE9928|   49668221|B000068O48|     734576678| 2015-08-31|
|R1P4RW1R9FDPEE|   12338275|B000GGKOG8|     614448099| 2015-08-31|
|R1EBPM82ENI67M|   38487968|B000NU4OTA|      72265257| 2015-08-31|
|R372S58V6D11AT|   23732619|B00JOQIO6S|     308169188| 2015-08-31|
|R1A4514XOYI1PD|   21257820|B008NCD2LG|     976385982| 2015-08-31|
|R20D9EHB7N20V6|    3084991|B00007FGUF|     670878953| 2015-08-31|
|R1WUTD8MVSROJU|    8153674|B00M9V2RMM|     508452933| 2015-08-31|
|R1QCYLT25812DM|   52246189|B00J3O9DYI|     766372886| 2015-08-31|
| R904DQPBCEM7A|   41463864|B00NS1A0E4|     458130381| 2015-08-31|
|R1DGA6UQIVLKZ7|    2781942|B007B5V092|     152891509| 2015-08-31|
| RLQT3V8SMNIBH|     707292|B00IODHGVG|     717335277| 2015-08

In [32]:
# Check for duplicates
review_id_df.groupby(['review_id']).count().where('count > 1').sort('count', ascending=False).show()

+---------+-----+
|review_id|count|
+---------+-----+
+---------+-----+



In [33]:
# Check data types
review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: string (nullable = true)



In [34]:
# to fit products table
products_df = df.select('product_id', 'product_title')
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00428R89M|yoomall 5M Antenn...|
|B000068O48|Hosa GPM-103 3.5m...|
|B000GGKOG8|Channel Master Ti...|
|B000NU4OTA|LIMTECH Wall char...|
|B00JOQIO6S|Skullcandy Air Ra...|
|B008NCD2LG|Pioneer SP-BS22-L...|
|B00007FGUF|C2G/Cables to Go ...|
|B00M9V2RMM|COOLEAD-HDMI Swit...|
|B00J3O9DYI|Philips Wireless ...|
|B00NS1A0E4|PlayStation 3 3D ...|
|B007B5V092|JVC HAFR201A Xtre...|
|B00IODHGVG|Sylvania Alarm Cl...|
|B0035PBHX6|Coby 8 GB 1.8-Inc...|
|B00K1JJWFO|Diamond (Original...|
|B00S3LJ5EA|Kingvom 8gb 50 Ho...|
|B00B5QNGN6|JBL Ultra-Portabl...|
|B00PLE8QF6|YIPBOWPT Surface ...|
|B00GHUSIU6|StarTech.com Mini...|
|B0045EJY90|TEAC CD-P650-B Co...|
|B003BT6BM8|Philips SHS8100/2...|
+----------+--------------------+
only showing top 20 rows



In [35]:
# Check for duplicates
products_df.groupby(['product_id']).count().where('count > 1').sort('count', ascending=False).show()

+----------+-----+
|product_id|count|
+----------+-----+
|B003L1ZYYM|15348|
|B0002L5R78|11168|
|B003EM8008| 9780|
|B0012S4APK| 9364|
|B0001FTVEK| 8798|
|B000WYVBR0| 7845|
|B0019EHU8G| 7586|
|B001HBHNHE| 7405|
|B00F5NE2KG| 6704|
|B004QK7HI8| 6538|
|B00D5Q75RC| 6075|
|B00316263Y| 5816|
|B0052SCU8U| 5763|
|B001GTT0VO| 5587|
|B002MAPT7U| 5297|
|B003WGRUQQ| 5073|
|B008KVUAGU| 5022|
|B001TH7GSW| 4872|
|B009OBCAW2| 4775|
|B004HHICKC| 4218|
+----------+-----+
only showing top 20 rows



In [36]:
# Drop duplicates
products_df = products_df.dropDuplicates(['product_id'])

In [38]:
# Check data types
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [54]:
products_df.count()

1231892

In [39]:
# to fit customers table
customers_df = df.select('customer_id')
customers_df = customers_df.groupBy('customer_id').count()
customers_df = customers_df.withColumnRenamed('count', 'customer_count')
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   42560427|             7|
|   28871066|             1|
|    6711334|             2|
|   46393060|             1|
|   48101124|             1|
|   38209321|             1|
|   47108763|             4|
|   43252205|             1|
|   41283385|             1|
|   46909180|             6|
|    5453476|             2|
|   12406466|             3|
|   48146680|             1|
|    5953456|             3|
|   16457323|             5|
|   24540309|             1|
|   43366878|             2|
|   28796717|             1|
|   28258386|             1|
|   26079415|             3|
+-----------+--------------+
only showing top 20 rows



In [40]:
# Check data types
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [55]:
customers_df.count()

4492104

In [41]:
# to fit vine table
vine_df = df.select('review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine')
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2MTG1GCZLR2DK|          5|            0|          0|   N|
|R2HBOEM8LE9928|          5|            0|          0|   N|
|R1P4RW1R9FDPEE|          5|            1|          1|   N|
|R1EBPM82ENI67M|          1|            0|          0|   N|
|R372S58V6D11AT|          5|            1|          1|   N|
|R1A4514XOYI1PD|          5|            1|          1|   N|
|R20D9EHB7N20V6|          5|            0|          0|   N|
|R1WUTD8MVSROJU|          5|            0|          0|   N|
|R1QCYLT25812DM|          4|            0|          0|   N|
| R904DQPBCEM7A|          4|            0|          0|   N|
|R1DGA6UQIVLKZ7|          5|            0|          0|   N|
| RLQT3V8SMNIBH|          5|            0|          0|   N|
|R3T9GZS2TMXZGM|          1|            0|          0|   N|
|R24HVAEYP5PLDN|          5|            

In [42]:
# Check for duplicates
vine_df.groupby(['review_id']).count().where('count > 1').sort('count', ascending=False).show()

+---------+-----+
|review_id|count|
+---------+-----+
+---------+-----+



In [43]:
# Check data types
vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [51]:

from pyspark.sql.types import IntegerType
# Change data type of column 'star_rating'
vine_df = vine_df.withColumn('star_rating', vine_df['star_rating'].cast(IntegerType()))

In [52]:
# Check data types
vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [56]:
vine_df.count()

7942659

## Load dataframes to the database

In [44]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-assignment.cmf64jlceuwo.us-west-1.rds.amazonaws.com:5432/bd_assignment_db"
config = {"user":"postgres", 
          "password": "iamwinner", 
          "driver":"org.postgresql.Driver"}

In [45]:
 # Write DataFrame to active_user table in RDS
review_id_df.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

In [46]:
products_df.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)

In [47]:
customers_df.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [53]:
vine_df.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)